## Binance Trading Bot in Python

The following Python script implements a mean reversion trading strategy to make profitable trades. A live price stream is fed in through the Binance WebSocket API which is stored in a continuously updated dataframe.

The script place a market buy order if the latest price (``last_price``) is above the n-second Simple Moving Average (SMA). Once it has an open position, it will place a market sell order if the latest price indicates a profit of ``target_profit`` or a loss of ``stop_loss``.

In [ ]:
%run ./../../.api_keys/binance_key.ipynb # import API key and secret

In [ ]:
from binance import Client
import websocket
import json
import pandas as pd

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
endpoint = 'wss://stream.binance.us:9443/ws' # Binance WebSocket API endpoint

In [ ]:
ticker = 'BTCUSDT'
our_msg = json.dumps({'method': 'SUBSCRIBE',
                      'params': [ticker.lower() + '@ticker'], 'id':1})

In [ ]:
# Initialise state variables
df = pd.DataFrame()
in_position = False

trade_amt = 25 # Initial trading amount, in this case, 25 USDT
target_profit = 0.002 # Targetted profit for exiting position
stop_loss = 0.002 # Stop loss for exiting position

def on_open(ws):
    ws.send(our_msg) # request ticker from server

def on_message(ws, message):
    global df, in_position, order_qty, buy_price
    n = 15
    out = json.loads(message)
    out = pd.DataFrame({'price':float(out['c'])}, index=[pd.to_datetime(out['E'], unit='ms')])
    df = pd.concat([df,out], axis=0)
    print(df)
    df = df.tail(n) # keep the latest n rows of data
    last_price = df.tail(1).price.values[0] # last row from data stream
    sma = df.price.rolling(n).mean().tail(1).values[0] # calculate the n-second SMA

    if not in_position and last_price > sma:
        order = client.create_order(symbol=ticker, side='BUY', type='MARKET', quoteOrderQty=trade_amt)
        order_qty = float(order['executedQty'])
        buy_price = float(order['fills'][0]['price'])
        in_position = True
        print(order)
    if in_position and (last_price > buy_price * (1 + target_profit) or last_price < buy_price * (1 - stop_loss)):
        order = client.create_order(symbol=ticker, side='SELL', type='MARKET', quantity=order_qty)
        in_position = False
        print(order)

In [ ]:
ws = websocket.WebSocketApp(endpoint, on_message=on_message, on_open=on_open)
ws.run_forever() # Run trading bot indefinitely